Это практически реальный код с соревнования. Эта часть кода создает новые фичи, обучает модель, получает результат

In [ ]:
import pandas as pd
import numpy as np
commongroups={'cinema':45,'food':67,'exhibition':73,'sport_events':81,'circus':88,'fitness':27,'theatre':32,
    'clubs':41}
commongroupslarge={'cinema':[45,2,6,32,33,39,53,56,58,82,7,89,78,89,16],'food':[67,0,6,11,56,58,61,74,110],
    'exhibition':[73,7,76,17,45,53,59,87,88,89,42,93,94,117,140],
    'sport_events':[81,18,27,55,33,78,77],'circus':[88,6,16,32,53,73,117,138,140,171],
    'fitness':[27,3,5,12,10,22,27,48,51,55,64,68,77,81,106,112,121,130,131,136,151,155,157,171,195,20],
    'theatre':[32,7,11,12,20,24,28,104,37,39,42,45,60,73,76,88,89,93,94,97,99,102,126,148,162,176,17],
    'clubs':[41,0,1,2,4,6,7,12,18,20,28,30,37,45,51,53,85,109,117,121,176]
};fortypical={32:55,41:125,45:68,67:45,73:58,81:38,88:83}
def getgroupfeatures(trans,idsvst,n):
    currentfeaturetrans=trans[trans['smallgroup']==idsvst[0]]
    del idsvst[0]
    for id in idsvst:
        currentfeaturetrans=pd.concat([currentfeaturetrans,trans[trans['smallgroup']==id]])
    del currentfeaturetrans['smallgroup'];del currentfeaturetrans['transdate']
    missingclients=(set(trans.client_dk)).difference(set(currentfeaturetrans.client_dk))
    if missingclients:
        missingvalues=pd.DataFrame([client,0,0,0,0,0,0]for client in missingclients)
        missingvalues.columns=['client_dk','mean','max','min','std','sum','count']
    res=currentfeaturetrans.groupby('client_dk')['amount'].agg(['mean','max','min','std','sum','count']).reset_index()
    if(missingclients):
        res=pd.concat([res,missingvalues])
    res.columns=[i if(i=='client_dk') else n+'_'+i for i in res.columns]
    return res
def aboutbreaks(trans,groupid):
    allusers=set(trans.client_dk)
    grouptrans=trans[trans.small_group==groupid]
    res=pd.DataFrame()
    for user in allusers:
        usersdates=grouptrans[grouptrans.client_dk==user]['trans_date']
        #сейчас будет одна из любимых строчек
        dop=np.array(sorted(list(usersdates))+[0])-np.array([0]+sorted(list(usersdates)));dop=dop[:len(dop) -1]
        if(len(dop)>1):
            res=pd.concat([res,pd.DataFrame(data=[[user,dop.mean(),np.median(dop),dop.std()]])])
        else:
            res=pd.concat([res,pd.DataFrame(data=[[user,-999,-999,-999]])])
    res.columns=['client_dk','time_mean_{}'.format(groupid),'time_median_{}'.format(groupid),'time_std_{}'.format(groupid)
    ]
    return res
def getdifusers(d,gr,st):
    typicalusers=train[train['small_group_{}'.format(gr)]>=st]
    typicaluser=pd.Series(np.array(typical_users).mean(axis=0))
    users=set(d.client_dk)
    res=pd.DataFrame()
    for user in users:
        npuser=np.array(d[d.client_dk==user]);np_user1=np.array(typicaluser)
        dist=np.power(np.power(np_user-np_user1,2).sum(),0.5)/len(np_user)/10**4
        res=pd.concat([res,pd.DataFrame(d=[[user,dist]])])
    res.columns=['client_dk','dist_group_{}'.format(gr)]
    return res
def getbaselinefeatures(trans):
    af=trans.groupby('client_dk')['amount'].agg(['mean','max','min','std','sum','count']).reset_index()
    counterdf=trans.groupby(['client_dk','small_group'])['amount'].count()
    catcounts=counterdf.reset_index().pivot(index='client_dk',columns='small_group',values='amount')
    catcounts=catcounts.fillna(0)
    catcounts.columns=['small_group_'+str(i) for i in catcounts.columns]
    return pd.merge(af,catcounts.reset_index(),on='client_dk')
def lastdate(trans,g):
    allusers=set(trans.client_dk)
    grouptrans=trans[trans.small_group==g]
    res=pd.DataFrame()
    for user in allusers:
        usersdates=grouptrans[grouptrans.client_dk==user]['trans_date'];dop=list(reversed(sorted(list(usersdates))))
        if(len(dop)>=1):
            res=pd.concat([res,pd.DataFrame(data=[[user,dop[0]]])]
            )
        else:
            res=pd.concat([res,pd.DataFrame(data=[[user,-999]])])
    res.columns=['client_dk','last_date_{}'.format(g)]
    return res
def getallfeatures(trans):
    lol=getbaselinefeatures(trans)
    for group in commongroups.keys():
        lol=pd.merge(lol,lastdate(trans,commongroups[group]),on='client_dk')
        lol=pd.merge(lol,aboutbreaks(trans,commongroups[group]),on='client_dk')
        lol=pd.merge(lol,getgroupfeatures(trans,commongroupslarge[group],group),on='client_dk')
    #да, я отвлекся и реально написал так:
    result=lol
    return result
traintrain=pd.read_csv('/content/data_final/transactions_train.csv')
traintarget=pd.read_csv('/content/data_final/train_target.csv')
vals=list(traintarget.columns[1:])
vals=[int(i)for i in vals]
def ilikeitbig(train,start,end,diff):
    datasets=[]
    for i in range(start,end,-diff):
        Xtr=train[train["trans_date"]<=i];ytr=train[train["trans_date"]>i];ytr=ytr[ytr["trans_date"]<=(i+7)]
        ytr=ytr[ytr["client_dk"].isin(Xtr["client_dk"].unique())]
        ytr=ytr[ytr["small_group"].isin([27,32,41,45,67,73,81,88])].groupby("client_dk").apply(lambda x:pd.Series([int(i in list(x['small_group'].values)) for i in vals])).reset_index()
        ytr.rename(columns={d:vals[d]for d in range(8)},inplace=True)
        datasets.append((Xtr,ytr))
    return datasets
alldatasets=ilikeitbig(traintrain,380,180,10)
alldatasets=[(getallfeatures(i),k)for i,k in alldatasets]
for ind in range(len(alldatasets)):
    alldatasets[ind][0]['client_dk']=alldatasets[ind][0]['client_dk'].astype('str')+'_'+str(ind)
    alldatasets[ind][1]['client_dk']=alldatasets[ind][1]['client_dk'].astype('str')+'_'+str(ind)
trtr=traintrain.copy()
trtr['client_dk']=trtr['client_dk'].astype('str')+'_-1'
tyty=train_target.copy()
tyty['client_dk']=tyty['client_dk'].astype('str')+'_-1'
mainX=trtr.copy()
mainy=tyty.copy()
for da,net in alldatasets:
    da.columns=[str(i) for i in da.columns]
    net.columns=[str(i) for i in net.columns]
    mainX=mainX.append(da,ignore_index=True,sort=False)
    mainy=mainy.append(net,ignore_index=True,sort=False)
mainXX=mainX[mainX['client_dk'].isin(mainy['client_dk'])]
mainXX=mainXX.sort_values(by='client_dk')
mainy=mainy.sort_values(by='client_dk')
mainXX=mainXX.reset_index().drop(columns=['index'])
mainy=mainy.reset_index().drop(columns=['index'])
testtrans=pd.read_csv('/content/drive/My Drive/transactions_test.csv')
test=getallfeatures(testtrans)
resultstree = {}
c=0
for q in traintarget.columns[1:]:
    currtargettrain=mainy.loc[:,q]
    model=CatBoostClassifier(iterations=800,depth=6,learning_rate=0.05,custom_metric='AUC',verbose=True)
    model.fit(mainXX.drop(columns=["client_dk"]),currtargettrain.values)
    pred=model.predict_proba(topredict.drop(columns=['client_dk']))[:,1]
    resultstree[q]=pred
    c+=1